In [1]:
import numpy as np

In [170]:
def integers(start_val, blk_size=100, use_negatives = False):
    x = np.arange(start_val, blk_size+start_val, dtype=np.float)
    while True:
        yield x
        if use_negatives:
            if start_val == 0:
                yield -x -1
            else:
                yield -x
        x += blk_size

In [171]:
gen = integers(1, use_negatives=True)
for i, x in enumerate(gen):
    print(x)
    if i == 3:
        break

[   1.    2.    3.    4.    5.    6.    7.    8.    9.   10.   11.   12.
   13.   14.   15.   16.   17.   18.   19.   20.   21.   22.   23.   24.
   25.   26.   27.   28.   29.   30.   31.   32.   33.   34.   35.   36.
   37.   38.   39.   40.   41.   42.   43.   44.   45.   46.   47.   48.
   49.   50.   51.   52.   53.   54.   55.   56.   57.   58.   59.   60.
   61.   62.   63.   64.   65.   66.   67.   68.   69.   70.   71.   72.
   73.   74.   75.   76.   77.   78.   79.   80.   81.   82.   83.   84.
   85.   86.   87.   88.   89.   90.   91.   92.   93.   94.   95.   96.
   97.   98.   99.  100.]
[  -1.   -2.   -3.   -4.   -5.   -6.   -7.   -8.   -9.  -10.  -11.  -12.
  -13.  -14.  -15.  -16.  -17.  -18.  -19.  -20.  -21.  -22.  -23.  -24.
  -25.  -26.  -27.  -28.  -29.  -30.  -31.  -32.  -33.  -34.  -35.  -36.
  -37.  -38.  -39.  -40.  -41.  -42.  -43.  -44.  -45.  -46.  -47.  -48.
  -49.  -50.  -51.  -52.  -53.  -54.  -55.  -56.  -57.  -58.  -59.  -60.
  -61.  -62.  -63.  -64. 

In [172]:
def f_s(s, w=1, l=1):
    return (s*w*(1-w)+l)/(s*(1-w)+l)

In [32]:
def exp_m_k(s, m, k, xed, yed):
    ek = (s*f_s(s) + k*k*np.pi*np.pi/xed/xed)**0.5
    return np.exp(-2*m*yed*ek)

In [108]:
m = np.arange(1,100)
s = 7
k = 1
xed = 20
yed = 10
a = exp_m_k(s, m, k, xed, yed)

In [173]:
def summator(f, start_val, block_size=10, axis=0, use_negatives=False, eps_rel=1e-6, max_iter=10):
    gen = integers(start_val, blk_size=block_size, use_negatives=use_negatives)
    inits = next(gen)
    total = np.sum(f(inits))
    if np.all(total == 0.):
        return total
    for i in range(max_iter):
        x = next(gen)
        diff = np.sum(f(x), axis=axis)
        rel_diff = np.abs(np.linalg.norm(diff/total))
        if rel_diff < eps_rel:
            return total + diff, rel_diff, i+1
        else:
            total += diff
    #raise RuntimeWarning("summator did no converge in {} steps".format(max_iter))
    return total, rel_diff, i

In [174]:
mm = np.ones((len(m), len(m)))*m

In [175]:
f = lambda x: exp_m_k(s, x, k, xed, yed)

In [176]:
summator(f, 1)

(2.6848983510153794e-12, 1.9466079002537945e-116, 1)

In [177]:
from numpy import cosh, sinh, sin, cos

In [178]:
def g_k(xi, xi_1, s, k, xd, xwd, xed, yd, ywd, yed):
    pi = np.pi
    yd1 = yed - np.abs(yd-ywd)
    yd2 = yed - (yd+ywd)
    u = s*f_s(s)
    ek = np.sqrt(u + k*k*pi*pi/xed/xed)
    return 2.*xed/k/pi*sin(k*pi*(xi_1-xi)/xed)*cos(k*pi/xed*(2.*xwd-xi-xi_1))*cos(k*pi*xd/xed)*(cosh(ek*yd1)+cosh(ek*yd2))/ek/sinh(ek*yed)

In [179]:
xends = 0.1*np.arange(-10,11)

In [197]:
s = 0.1
xed = 10.
yed = 5.
xwd = xed/2.
ywd = yed/2.
xd = xwd
yd = ywd

In [198]:
func = lambda k_: g_k(xends[0], xends[1], s, k_, xd, xwd, xed, yd, ywd, yed)

In [199]:
summator(func, 1, max_iter=40)

(-0.15205841538222017, 0.00017710640741620611, 39)

In [141]:
f_s(s)

1.0